#FRETBursts - ns-ALEX example

*This notebook is part of a [tutorial series](https://github.com/tritemio/FRETBursts_notebooks) for the [FRETBursts](http://tritemio.github.io/FRETBursts/) burst analysis software.*

> For a step-by-step introduction to FRETBursts usage please refer to [us-ALEX smFRET burst analysis](FRETBursts - us-ALEX smFRET burst analysis.ipynb).
>
> In this notebook we present a typical [FRETBursts](https://github.com/tritemio/FRETBursts/)
> workflow for **ns-ALEX smFRET burst analysis**. 

*While FRETBursts does not specifically includes functions for fitting TCSPC fluorescence decays, a fitting with exponetial decays and IRF deconvolution can be easily performed using standard python libraries. For an example and a brief discussion see the notebook [Lifetime decay fit](http://nbviewer.ipython.org/github/tritemio/notebooks/blob/master/Lifetime_decay_fit.ipynb).*

## Loading FRETBursts

In [ ]:
from fretbursts import *

In [ ]:
sns = init_notebook()

## Selecting a data file

The following command will an open-file dialog in order to select a data file:

In [ ]:
fname = OpenFileDialog()
fname

## Load the selected file

Here we load the file and we set the alternation parameters:

In [ ]:
d = loader.nsalex(fname)

In [ ]:
d.time_max

In [ ]:
d.det_t

In [ ]:
np.unique(d.det_t)

In [ ]:
d.add(D_ON=(200, 1500), A_ON=(1750, 3200), det_donor_accept=(6, 4))

In [ ]:
bpl.plot_alternation_hist(d)

Execute the previous 2 cells untill you get a satisfying selection for the excitation periods. Then run the following to apply the parameters:

In [ ]:
loader.alex_apply_period(d)

##Burst search and selection

In [ ]:
d.calc_bg(fun=bg.exp_fit, time_s=30, tail_min_us='auto', F_bg=1.7)

In [ ]:
dplot(d, timetrace_bg)

In [ ]:
dplot(d, timetrace)
xlim(1, 2)
ylim(-50, 50)

In [ ]:
d.burst_search()

In [ ]:
ds = d.select_bursts(select_bursts.size, th1=30)

In [ ]:
alex_jointplot(ds)

In [ ]:
ds.leakage = 0.09

In [ ]:
alex_jointplot(ds)

In [ ]:
dplot(ds, hist_fret, show_kde=True)

## Nano-time

The nanotime for the measurement is saved in the `.nanotime` attribute of the `Data()` variable (here either `d` or `ds`).

As an example here we get the array of nanotimes for all photons, donor emission and acceptor emsission:

In [ ]:
nanotimes_t = d.nanotimes
nanotimes_d = d.nanotimes[d.get_D_em()]
nanotimes_a = d.nanotimes[d.get_A_em()]

We can plot the histogram for this 3 nanotimes:

In [ ]:
hist_params = dict(bins=range(4096), histtype='step', alpha=0.6, lw=1.5)
hist(nanotimes_t, color='k', label='Total ph.', **hist_params)
hist(nanotimes_d, color='g', label='D. em. ph.', **hist_params)
hist(nanotimes_a, color='r', label='A. em. ph.', **hist_params)
plt.legend()
#xlim(0, 2000)

We can also select only nanotimes of photons inside bursts. Here, as an example, we will use the `ds` variable that contains a selection of bursts.

First we compute a selection mask (a boolean array) for photons inside bursts:

In [ ]:
ph_in_bursts_mask = bl.ph_select(ds.ph_times_m[0], ds.mburst[0])

Then we apply this selection to the nanotimes array. To get the donor- and acceptor-emission nanotimes we combine the in-bursts selection mask (`ph_in_bursts_mask`) with the donor or acceptor emission mask (that we get with `.get_D_em()` and `.get_D_em()`):

In [ ]:
bursts_nanotimes_t = d.nanotimes[ph_in_bursts_mask]
bursts_nanotimes_d = d.nanotimes[ph_in_bursts_mask * d.get_D_em()]
bursts_nanotimes_a = d.nanotimes[ph_in_bursts_mask * d.get_A_em()]

And, as before, we can histogram the nanotimes:

In [ ]:
hist_params = dict(bins=range(4096), histtype='step', alpha=0.6, lw=1.5)
hist(bursts_nanotimes_t, color='k', label='Total ph.', **hist_params)
hist(bursts_nanotimes_d, color='g', label='D. em. ph.', **hist_params)
hist(bursts_nanotimes_a, color='r', label='A. em. ph.', **hist_params)
plt.legend()
xlim(0, 2000)

##Saving to a file

Saving some of all these arrays to file is straightforward.

### Save array to txt comma-separed-values

To save a single array to a file we can use the [`.tofile`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.tofile.html) method:

In [ ]:
with open('nanotimes_t.txt', 'w') as f:
    nanotimes_t.tofile(f, sep=',\n')  # save in CSV txt format

### Save to MATLAB format

To save a set of arrays in MATLAB format  we can use the [`scipy.io.savemat`](http://docs.scipy.org/doc/scipy-0.13.0/reference/generated/scipy.io.savemat.html) function.

Here we save 3 arrays `bursts_nanotimes_t`, `bursts_nanotimes_d` and `bursts_nanotimes_a` to a file called *bursts_nanotimes.mat*:

In [ ]:
from scipy.io import savemat

In [ ]:
savemat('bursts_nanotimes.mat', 
        dict(bn_t=bursts_nanotimes_t, 
             bn_d=bursts_nanotimes_d, 
             bn_a=bursts_nanotimes_a,))

When loaded in MATLAB the arrays will be named `bn_t`, `bn_d` and `bn_a`.

#Notebook style

In [ ]:
from IPython.core.display import HTML
HTML(open("./styles/custom.css", "r").read())